In [ ]:
import numpy as np
import random

In [ ]:
def load_movielens(path='./ml-100k'):
    # get movie titles
    movies = {}
    for line in open(path + '/u.item', encoding='latin-1'):
        id, title = line.split('|')[0:2]
        movies[id] = id
    # load data
    prefs = {}
    for line in open(path + '/u.data', encoding='latin-1'):
        user, movieid, rating, ts = line.split('\t')
        prefs.setdefault(user, {})
        prefs[user][movies[movieid]] = float(rating)
    return prefs

In [ ]:
prefs = load_movielens()
print(prefs['88'])

In [ ]:
def split_data(data, M, k, seed):
    test = []
    train = []
    random.seed(seed)
    for user in data:
        if random.randint(0, M) == k:
            test.append([user, data[user]])
        else:
            train.append([user, data[user]])
    return train, test

In [ ]:
train, test = split_data(prefs, 8, 1, 1)
def changement(data):
    x = []
    y = []
    for i in range(len(data)):
        for j in range(1682):
            x.append(data[i][1].get(str(j+1), 0.0))
        y.append(x)
        x = []
    y = np.array(y)
    return y
train = changement(train)
test = changement(test)

In [ ]:
def cos_sim(x ,y):
    numerator = x * y.T
    NM = len(numerator[numerator>0])
    N = np.sqrt(x * x.T)
    N = len(N[N>0])
    M = np.sqrt(y * y.T)
    M = len(M[M>0])
    denominator = M * N
    return NM / denominator  

In [ ]:
def similarity(data):
    m = np.shape(data)[0]
    w = np.mat(np.zeros((m, m)))
    for i in range(m):
        for j in range(i, m):
            if not j == i:
                w[i, j] = cos_sim(data[i], data[j])
                w[j, i] = w[i, j]
            else:
                w[i, j] = 0
    return w

In [ ]:
w = similarity(train)

In [ ]:
def user_based_recommend(data, w, user):
    m, n = np.shape(data)
    interaction = data[user, ]
    not_inter = []
    for i in range(n):
        if interaction[i] == 0:
            not_inter.append(i)
    predict = {}
    for x in not_inter:
        item = np.copy(data[:, x])
        for i in range(m):
            if item[i] != 0:
                if x not in predict:
                    predict[x] = w[user, i] * item[i]
                else:
                    predict[x] = predict[x] + w[user, i] + item[i]
    return sorted(predict.items(), key = lambda d: d[1], reverse=True)
predict = user_based_recommend(train, w, 88)
predict

In [ ]:
# top_n 
def top_k(predict, n):
    top_recom = []
    len_result = len(predict)
    if n>= len_result:
        top_recom = predict
    else:
        for i in range(n):
            top_recom.append(predict[i])
    return top_recom
top_k(predict, 5)